# Image Generation with AutoEncoders

In [10]:
from data import CelebADataset
from models import VariationalAutoEncoder as VAE, WGAN
from tensorflow.keras.optimizers.legacy import Adam
from utils import plot_history
import tensorflow as tf
train, val, test = map(CelebADataset, ('train', 'val', 'test'))

## Table of Contents

## Data preprocessing

We followed the CelebA dataset [official split](https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) to create a train (80%), validation (10%) and test (10%) subsets. In order to create the same split, the script [data.py](data.py) automatically prepares the dataset subfolders to load the images (it is required to previously download the [archive.zip](https://www.kaggle.com/datasets/jessicali9530/celeba-dataset) from Kaggle).

## Baseline 

### Variational AutoEncoder 

In [4]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()
vae = VAE(CelebADataset.IMG_SIZE, hidden_size = 200, filters = [16, 32, 32, 32], kernels = [3,3,3,3], strides = [2,2,2,2])
vae_history = vae.train(train, val, test, 'results/vae/', optimizer=Adam(1e-4), epochs=100)

Found 162770 files belonging to 1 classes.
Found 19867 files belonging to 1 classes.


2024-05-14 19:47:29.474748: W tensorflow/c/c_api.cc:305] Operation '{name:'count_7/Assign' id:9654 op device:{requested: '', assigned: ''} def:{{{node count_7/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](count_7, count_7/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-05-14 19:47:31.476679: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: type mismatch for node 'DatasetToGraphV2_1': expected a subtype of:
type_id: TFT_PRODUCT
args {
  type_id: TFT_ENCODED
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_FLOAT
        }
 

Train on 16277 steps, validate on 1987 steps
Epoch 1/20
16269/16277 [============================>.] - ETA: 0s - batch: 8134.0000 - size: 1.0000 - loss: 48.1182 - r_loss: 0.0389 - kl_loss: 9.1947

/home/ana/Documents/dl-labs/P3/.venv/lib/python3.10/site-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2024-05-14 19:48:36.044587: W tensorflow/c/c_api.cc:305] Operation '{name:'loss_1/mul' id:9724 op device:{requested: '', assigned: ''} def:{{{node loss_1/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss_1/mul/x, loss_1/decoder_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


16277/16277 [==============================] - 65s 4ms/step - batch: 8138.0000 - size: 1.0000 - loss: 48.1158 - r_loss: 0.0389 - kl_loss: 9.1949 - val_loss: 41.7929 - val_r_loss: 0.0324 - val_kl_loss: 9.4175
Epoch 2/20
16277/16277 [==============================] - 62s 4ms/step - batch: 8138.0000 - size: 1.0000 - loss: 41.2132 - r_loss: 0.0314 - kl_loss: 9.8537 - val_loss: 40.5121 - val_r_loss: 0.0303 - val_kl_loss: 10.1656
Epoch 3/20
16277/16277 [==============================] - 62s 4ms/step - batch: 8138.0000 - size: 1.0000 - loss: 40.4757 - r_loss: 0.0304 - kl_loss: 10.1117 - val_loss: 40.0038 - val_r_loss: 0.0299 - val_kl_loss: 10.0622
Epoch 4/20
16277/16277 [==============================] - 63s 4ms/step - batch: 8138.0000 - size: 1.0000 - loss: 40.1617 - r_loss: 0.0300 - kl_loss: 10.1811 - val_loss: 39.8622 - val_r_loss: 0.0299 - val_kl_loss: 9.9539
Epoch 5/20
16277/16277 [==============================] - 63s 4ms/step - batch: 8138.0000 - size: 1.0000 - loss: 39.9758 - r_loss: 

/home/ana/Documents/dl-labs/P3/.venv/lib/python3.10/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2024-05-14 20:08:31.647978: W tensorflow/c/c_api.cc:305] Operation '{name:'decoder_1/activation_95/Sigmoid' id:9634 op device:{requested: '', assigned: ''} def:{{{node decoder_1/activation_95/Sigmoid}} = Sigmoid[T=DT_FLOAT, _has_manual_control_dependencies=true](decoder_1/cropping2d_7/strided_slice)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-05-14 20:09:02.098445: W tensorflow/c/c_api.cc:305] Operation '{name:'batch_normalization_187/beta/Assign' id:15767 op device:{requested: '', assigned: ''} def:{{{node batch_normaliza

In [30]:
plot_history(vae_history, 'loss').show()
plot_history(vae_history, 'kl_loss').show()
plot_history(vae_history, 'r_loss').show()

### WGAN-GP

In [2]:
wgan = WGAN(CelebADataset.IMG_SIZE, hidden_size = 128, critic_steps=3, gp_weight=10)
wgan_history = wgan.train(train, test, './results/wgan/', epochs=10, batch_size=20)

Found 162770 files belonging to 1 classes.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unknown node type <gast.gast.Assign object at 0x7eaf3fffdba0>
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unknown node type <gast.gast.Assign object at 0x7eaf3fffdba0>
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/10


2024-05-15 00:56:41.025226: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape indiscriminator/dropout/dropout_1/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-05-15 00:56:41.639885: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-05-15 00:56:43.267302: I external/local_xla/xla/service/service.cc:168] XLA service 0x7eadec022400 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-15 00:56:43.267348: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-05-15 00:56:43.267361: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-05-15 00:56:43.277634: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR cr

1500/1500 [==============================] - 67s 39ms/step - c_loss: -20.3939 - c_wass_loss: -26.3396 - c_gp: 0.5946 - g_loss: -33.5623 - c_acc: 69.2472 - g_acc: 39.1306
Epoch 2/10
1500/1500 [==============================] - 59s 39ms/step - c_loss: -5.5234 - c_wass_loss: -7.2934 - c_gp: 0.1770 - g_loss: -27.5093 - c_acc: 47.5217 - g_acc: 54.8668
Epoch 3/10
1500/1500 [==============================] - 59s 40ms/step - c_loss: -1.5666 - c_wass_loss: -4.2019 - c_gp: 0.2635 - g_loss: 378.3404 - c_acc: 19.8301 - g_acc: 80.6296
Epoch 4/10
1500/1500 [==============================] - 59s 40ms/step - c_loss: -5.1241 - c_wass_loss: -6.0595 - c_gp: 0.0935 - g_loss: 170.4925 - c_acc: 33.5610 - g_acc: 67.1319
Epoch 5/10
1500/1500 [==============================] - 59s 39ms/step - c_loss: -1.8228 - c_wass_loss: -2.7776 - c_gp: 0.0955 - g_loss: 231.4883 - c_acc: 20.2765 - g_acc: 80.4530
Epoch 6/10
1500/1500 [==============================] - 27s 18ms/step - c_loss: -2.7279 - c_wass_loss: -3.5106 - c

In [9]:
plot_history(wgan_history, name=['c_acc', 'g_acc']).update_xaxes(title_text='acc')